**Script for Concatenating Apache JMeter Test Results**

The following Python code was used to concatenate the results outputted by Apache JMeter. 

*NB: This script was not part of the main project but was employed solely for the purpose of efficiently gathering and organizing the test results from our JMeter performance tests.*

In [4]:
import json
import pandas as pd

In [5]:
soap_sum_res_time = 0
soap_sum_throughput = 0
soap_count = 0

graphql_sum_res_time = 0
graphql_sum_throughput = 0
graphql_count = 0

rest_sum_res_time = 0
rest_sum_throughput = 0
rest_count = 0

with open("res.json", "r") as file:
    data = json.load(file)
    for key in data:
        if "SOAP" in key:
            soap_count += 1
            soap_sum_res_time += data[key]["meanResTime"]
            soap_sum_throughput += data[key]["throughput"]
        if "GraphQL" in key:
            graphql_count += 1
            graphql_sum_res_time += data[key]["meanResTime"]
            graphql_sum_throughput += data[key]["throughput"]
        if "REST" in key:
            rest_count += 1
            rest_sum_res_time += data[key]["meanResTime"]
            rest_sum_throughput += data[key]["throughput"]
    
    soap_mean_res_time = soap_sum_res_time/soap_count
    soap_mean_throughput = soap_sum_throughput/soap_count

    graphql_mean_res_time = graphql_sum_res_time/graphql_count
    graphql_mean_throughput = graphql_sum_throughput/graphql_count
    
    rest_mean_res_time = rest_sum_res_time/rest_count
    rest_mean_throughput = rest_sum_throughput/rest_count

df = pd.DataFrame({
    "REST Mean Response": [rest_mean_res_time], 
    "SOAP Mean Response": [soap_mean_res_time], 
    "GraphQL Mean Response": [graphql_mean_res_time],
    "REST Mean Throughput": [rest_mean_throughput],
    "SOAP Mean Throughput": [soap_mean_throughput], 
    "GraphQL Mean Throughput": [graphql_mean_throughput]
}, index=["Mean Values"])

df

,REST Mean Response,SOAP Mean Response,GraphQL Mean Response,REST Mean Throughput,SOAP Mean Throughput,GraphQL Mean Throughput
Mean Values,14118.738864,17208.91019,15895.107065,0.400215,0.330517,0.354925


In [6]:
df = pd.read_csv("res.csv")
df['api_type'] = df['label'].apply(lambda x: 'REST' if 'REST' in x else 
                                   ('SOAP' if 'SOAP' in x else 
                                    ('GraphQL' if 'GraphQL' in x  else 'Unknown')))
average_latency = df.groupby('api_type')['Latency'].mean()

df_errors = df[(df['responseCode'].astype(str).str.startswith('5')) | (df['responseCode'].astype(str).str.startswith('4')) | (~df['failureMessage'].isna())]
df_errors
api_type_counts = df.groupby('api_type').size()
api_type_error_counts = df_errors.groupby('api_type').size()

# df = pd.DataFrame({
#     # "GraphQL Error Rate %": [(api_type_error_counts.loc['GraphQL']/api_type_counts.loc['GraphQL'])*100], 
#     "SOAP Error Rate %": [(api_type_error_counts.loc['SOAP']/api_type_counts.loc['SOAP'])*100], 
#     "REST Error Rate %": [(api_type_error_counts.loc['REST']/api_type_counts.loc['REST'])*100]
# }, index=["Error Rates"])
# api_type_error_counts
# api_type_error_counts
average_latency


api_type
GraphQL     5385.010000
REST        3532.406000
SOAP        4402.944500
Unknown    35520.961333
Name: Latency, dtype: float64